## Classification Models

In [2]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

from imblearn.over_sampling import SMOTE

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
X_train = pd.read_csv("../Data/X_train_default.csv").iloc[:,2:176]
y_train = pd.read_csv("../Data/y_train_default.csv").iloc[:,1]
X_test = pd.read_csv("../Data/X_test_default.csv").iloc[:,2:176]
y_test = pd.read_csv("../Data/y_test_default.csv").iloc[:,1]

In [4]:
X_train = X_train.drop(columns=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'])
X_test = X_test.drop(columns=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'])

In [5]:
smote = SMOTE(sampling_strategy='auto', random_state=42, n_jobs=-1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
X_test_resampled, y_test_resampled = smote.fit_resample(X_test, y_test)

### Logistic Regression

In [14]:
parameters_logistic_regression = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter': [1000]
}

grid_search = GridSearchCV(LogisticRegression(), parameters_logistic_regression, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_predictions = best_model.predict(X_test_resampled)
best_conf_matrix = confusion_matrix(y_test_resampled, best_predictions)

/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:767

In [1]:
roc_auc_score(y_test_resampled, best_predictions)

NameError: name 'roc_auc_score' is not defined

In [16]:
best_score

0.9572673236327395

In [17]:
tn, fp, fn, tp = confusion_matrix(y_test_resampled, best_predictions).ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

In [20]:
best_params

{'C': 0.01, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}

In [18]:
sensitivity

0.9006260611205433

In [19]:
specificity

1.0

### CART

In [25]:
parameters_CART = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [10,20,30],
    'min_samples_leaf': [10,20,30],
}

grid_search_CART = GridSearchCV(DecisionTreeClassifier(), parameters_CART, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search_CART.fit(X_train_resampled, y_train_resampled)
best_CART_model = grid_search_CART.best_estimator_
best_params = grid_search_CART.best_params_
best_score = grid_search_CART.best_score_
best_predictions = best_CART_model.predict(X_test_resampled)
best_conf_matrix = confusion_matrix(y_test_resampled, best_predictions)

/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/validation.py:767

In [26]:
roc_auc_score(y_test_resampled, best_predictions)

0.9180284380305603

In [27]:
tn, fp, fn, tp = confusion_matrix(y_test_resampled, best_predictions).ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

In [28]:
specificity

0.9850382003395586

In [29]:
best_score

0.9504199477602804

### Output from best models

In [8]:
# Instantiate the model with your specific parameters
log_model = LogisticRegression(C=0.01, max_iter=1000, penalty='l2', solver='newton-cg')

# Fit the model to your resampled training data
log_model.fit(X_train_resampled, y_train_resampled)

/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/scipy/optimize/_linesearch.py:425: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/janphilippgirgott/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


LogisticRegression(C=0.01, max_iter=1000, solver='newton-cg')

In [14]:
log_preds = log_model.predict_proba(X_test_resampled)
log_preds = log_preds[:,1]

In [20]:
CART_preds = best_CART_model.predict_proba(X_test_resampled)
CART_preds = CART_preds[:,1]

In [21]:
combined_preds = pd.DataFrame({'log_preds': log_preds, 'CART_preds': CART_preds})

In [24]:
combined_preds.to_csv("combined_preds_log_CART.csv")